In [13]:
import json
import pandas as pd

In [8]:
## 호텔 리뷰 데이터 바로 열면 컴퓨터 다운되기 때문에 열지 않기
with open('./data/hotels_korea_reviews_text.json', 'r', encoding='utf-8') as f:
    data = f.read()

In [9]:
data = json.loads(data)

# 호텔 인포 테이블
----------------------------------------------------------
- 호텔 정보 테이블    
    - hotelId
    - 호텔이름
    - 호텔등급 
    - 총리뷰수
    - 평점
    - 청결도 "cleanliness":"8.0",
    - 서비스 "service":"7.6",
    - 편안함 "comfort":"8.2",
    - 시설상태 "condition":"7.8",
    - 주변환경 "neighbourhood":"8.2"
    - 지역 region
    - 시도 locality

----------------------------------------------------------
- 사용자리뷰 테이블
    - 리뷰일
    - 고객id "itineraryId"
    - 여행타입
    - 평점
    - 리뷰 text


## 호텔 정보 테이블

In [118]:
len(data)

9027

In [272]:
data[0]['data']['body']['propertyDescription']['address']['locality']

'거제시'

hotelId : data['data']['body']['pdpHeader']['hotelId']<br>
호텔이름 : data['data']['body']['propertyDescription']['name']<br>
호텔등급 : data['data']['body']['propertyDescription']['starRatingTitle']<br>
지역 region : data['data']['body']['propertyDescription']['region']<br>
시도 locality : data['data']['body']['propertyDescription']['locality']<br>
총리뷰수 : data['data']['body']['reviewContent']['overall']['totalCount']<br>
평점 : data['data']['body']['reviewContent']['overall']['rating'][<br>
청결도 : data['data']['body']['reviewContent']['overall']['ratingAspects']['cleanliness']<br>
서비스 : data['data']['body']['reviewContent']['overall']['ratingAspects']['service']<br>
편안함 : data['data']['body']['reviewContent']['overall']['ratingAspects']['comfort']<br>
시설상태 : data['data']['body']['reviewContent']['overall']['ratingAspects']['condition']<br>
주변환경  : data['data']['body']['reviewContent']['overall']['ratingAspects']['neighbourhood']<br>
 

In [402]:
hotel = {}
for i in range(len(data)):
    for key in data[i]['data']['body']:
#         print(key)
        if key == 'pdpHeader':
            for key2, value in data[i]['data']['body'][key].items():
#                 print(key2, value)
                if key2 == 'hotelId':
                    hotel.setdefault(key2, []).append(value)
#             print(hotel)
        elif key == 'propertyDescription':
            for key2, value in data[i]['data']['body'][key].items():
#                 print(key2, value)
                if key2 in ('name','starRatingTitle'):
                    hotel.setdefault(key2, []).append(value)
                elif key2 == 'address':
                    for key3, value2 in data[i]['data']['body'][key][key2].items():
                        if key3 in ('region', 'locality'):
                            hotel.setdefault(key3, []).append(value2)
        elif key == 'reviewContent':
            for key2 in data[i]['data']['body'][key]:
#                 print(key2)
                if key2 == 'overall':
                    for key3, value in data[i]['data']['body'][key][key2].items():
                        if key3 in ('totalCount','rating'):
                            hotel.setdefault(key3, []).append(value)
                        elif key3 == 'ratingAspects':
#                             print(key3)
                            for key4, value3 in data[i]['data']['body'][key][key2][key3].items():
#                                 print(key4, value3)
                                hotel.setdefault(key4, []).append(value3)
    
    for key in hotel.keys():
        if len(hotel[key]) < i + 1:
            hotel[key].append(None)
    
#     print(f"{i+1} / {len(data)} 페이지 데이터 추출 중 입니다.")
    
else:
    print(f"{i+1} / {len(data)} 페이지 데이터 추출 완료되었습니다.")

                

9027 / 9027 페이지 데이터 추출 완료되었습니다.


In [403]:
# 각 키의 리스트 길이 출력
print("Hotel 딕셔너리 상태:")
for key, value in hotel.items():
    print(f"{key}: 길이 {len(value)}")

Hotel 딕셔너리 상태:
hotelId: 길이 9027
locality: 길이 9027
region: 길이 9027
name: 길이 9027
starRatingTitle: 길이 9027
rating: 길이 9027
totalCount: 길이 9027
cleanliness: 길이 9027
service: 길이 9027
comfort: 길이 9027
condition: 길이 9027
neighbourhood: 길이 9027


In [404]:
import pandas as pd
hotel_df = pd.DataFrame(hotel)
hotel_df

,hotelId,locality,region,name,starRatingTitle,rating,totalCount,cleanliness,service,comfort,condition,neighbourhood
0,1041793024,거제시,경상남도,상상속의집 (SangSang Hotel),3성급,7.8,67,8.0,7.6,8.2,7.8,8.2
1,1041793024,거제시,경상남도,상상속의집 (SangSang Hotel),3성급,7.8,67,8.0,7.6,8.2,7.8,8.2
2,1041793024,거제시,경상남도,상상속의집 (SangSang Hotel),3성급,7.8,67,8.0,7.6,8.2,7.8,8.2
3,589827,서귀포시,제주특별자치도,바다마루 제주 (Badamaru Jeju),2.5성급,8.4,20,9.0,8.4,8.6,8.4,5.0
4,589827,서귀포시,제주특별자치도,바다마루 제주 (Badamaru Jeju),2.5성급,8.4,20,9.0,8.4,8.6,8.4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9022,507669,서귀포시,제주특별자치도,호텔 토스카나 (Hotel Toscana),4성급,9.2,608,9.2,9.2,9.4,9.2,7.8
9023,507669,서귀포시,제주특별자치도,호텔 토스카나 (Hotel Toscana),4성급,9.2,608,9.2,9.2,9.4,9.2,7.8
9024,718356256,평창군,강원도,평창펜션하늘숲 (Pyeongchang The Sky Forest),3성급,7.4,3,8.6,8.0,6.6,7.4,6.0
9025,718356256,평창군,강원도,평창펜션하늘숲 (Pyeongchang The Sky Forest),3성급,7.4,3,8.6,8.0,6.6,7.4,6.0


In [406]:
import string

user = {}
for i in range(len(data)):
    for key in data[i]['data']['body']:
#         print(key)
#         if key == 'pdpHeader':
#             for key2, value in data[i]['data']['body'][key].items():
# #                 print(key2, value)
#                 if key2 == 'hotelId':
#                     user.setdefault(key2, []).append(value)
        if key == 'reviewContent':
#             print(data[i]['data']['body'][key])
            for group in data[i]['data']['body'][key]['reviews']['hermes']['groups']:
                # print(group)
                temp = {}
                for item in  group['items']:
#                     print(item)
                    for key, value in item.items():
#                        if key in ('reviewDate', 'itineraryId', 'tripType', 'rating', 'description'):
                            temp[key] = value if key != 'description' else (
                                str(value).translate(str.maketrans("", "", string.punctuation))
                                .replace("  ", " ").replace("\n", " ").replace("\r", " ").replace(",", "")
                            )
                    # 모든 필드를 user에 추가 (누락된 경우 None)
                    for field in ['reviewDate', 'itineraryId', 'tripType', 'rating', 'description']:
                        user.setdefault(field, []).append(temp.get(field, None))
# print(result)
#     print(f"{i+1} / {len(data)} 페이지 데이터 추출 중 입니다.")
    
else:
    print(f"{i+1} / {len(data)} 페이지 데이터 추출 완료되었습니다.")


9027 / 9027 페이지 데이터 추출 완료되었습니다.


In [407]:
# 각 키의 리스트 길이 출력
print("Hotel 딕셔너리 상태:")
for key, value in user.items():
    print(f"{key}: 길이 {len(value)}")

Hotel 딕셔너리 상태:
reviewDate: 길이 208485
itineraryId: 길이 208485
tripType: 길이 208485
rating: 길이 208485
description: 길이 208485


In [408]:
# 아이디와 맞추기 위해서는 길이가 같아야 하는데 외국 리뷰까지 포함되면 길이가 같아질 수 없어서 아이디 제거
import pandas as pd
user_review_df = pd.DataFrame(user)
user_review_df

,reviewDate,itineraryId,tripType,rating,description
0,2020년 11월 20일,72009037635503,family,10.0,주차가 객실당 한칸씩 차고가 있는게 굉장히 특이했으며 가장 편리했음 간혹 차에 무언...
1,2020년 11월 14일,9202656047317,family,10.0,1 오션뷰 남해안의 어느 숙박업소가 그렇듯이 오션뷰가 좋습니다 이 호텔앞의 바다...
2,2020년 8월 11일,9196559102828,other,10.0,커플이 다녀가기에 좋은 숙소 프라이빗함과 뷰도 갖춘 좋은 곳
3,2020년 8월 10일,9201554317978,other,8.0,위치와 뷰는 정말 좋습니다 개별주차장이 생각보다 쓰기 좋았습니다 다만 호텔보다는 고...
4,2020년 8월 7일,9196473456283,family,8.0,스탠다드 이용했어요 방마다 차고가 있어서 편하게 짐 옮겼네요 제가 이용한 방에 티비...
...,...,...,...,...,...
208480,2019년 1월 19일,8141646862293,family,8.0,Pyeongchang The Sky Forest Review Service We...
208481,2018년 12월 29일,8159891951288,family,6.0,The checkin took some time as there were none ...
208482,2020년 9월 14일,9196902905665,business,8.0,
208483,2020년 7월 13일,9201269638204,business,2.0,


## Hotel info 수집하기

In [370]:
# 호텔ID
hotel_id = data[0]['data']['body']['pdpHeader']['hotelId']

In [371]:
# 호텔이름
hotel_name = data[0]['data']['body']['propertyDescription']['name']

In [372]:
# 호텔 등급
hotel_grade = data[0]['data']['body']['propertyDescription']['starRating'] 

In [373]:
# 호텔 지역
hotel_region = data[0]['data']['body']['propertyDescription']['address']['region']

In [374]:
# 호텔 주소 시도
hotel_sido = data[0]['data']['body']['propertyDescription']['address']['locality']

In [377]:
# 총리뷰수
n_reviews = data[0]['data']['body']['reviewContent']['overall']['totalCount']

In [378]:
# 평점
avg_rating = data[0]['data']['body']['reviewContent']['overall']['rating']

In [383]:
# 세부평점 청결도
cleanliness = data[0]['data']['body']['reviewContent']['overall']['ratingAspects']['cleanliness']
service = data[0]['data']['body']['reviewContent']['overall']['ratingAspects']['service']
comfort = data[0]['data']['body']['reviewContent']['overall']['ratingAspects']['comfort']
condition = data[0]['data']['body']['reviewContent']['overall']['ratingAspects']['condition']
neighbourhood = data[0]['data']['body']['reviewContent']['overall']['ratingAspects']['neighbourhood']

In [384]:
print(cleanliness, service, comfort, condition, neighbourhood)

8.0 7.6 8.2 7.8 8.2


In [386]:
hotel_info_result = {}
for idx, hotel in enumerate(data):
#     print(f"{idx+1}/{len(data)}째 페이지 수집중")
    hotel_id = hotel['data']['body']['pdpHeader']['hotelId']
    hotel_name = hotel['data']['body']['propertyDescription']['name']
    hotel_grade = hotel['data']['body']['propertyDescription']['starRating']
    n_reviews = hotel['data']['body']['reviewContent']['overall']['totalCount']
    avg_rating = hotel['data']['body']['reviewContent']['overall']['rating']
    cleanliness = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('cleanliness', 0.0)
    service = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('service', 0.0)
    comfort = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('comfort', 0.0)
    condition = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('condition', 0.0)
    neighbourhood = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('neighbourhood', 0.0)
    hotel_sido = hotel['data']['body']['propertyDescription']['address']['locality']
    hotel_region = hotel['data']['body']['propertyDescription']['address'].get('region') if hotel_sido != '서울특별시' else '서울특별시'
    

    hotel_info_result.setdefault("hotel_id", []).append(hotel_id)
    hotel_info_result.setdefault("hotel_name", []).append(hotel_name)
    hotel_info_result.setdefault("hotel_grade", []).append(hotel_grade)
    hotel_info_result.setdefault("n_reviews", []).append(n_reviews)
    hotel_info_result.setdefault("avg_rating", []).append(avg_rating)
    hotel_info_result.setdefault("cleanliness", []).append(cleanliness)
    hotel_info_result.setdefault("service", []).append(service)
    hotel_info_result.setdefault("comfort", []).append(comfort)
    hotel_info_result.setdefault("condition", []).append(condition)
    hotel_info_result.setdefault("neighbourhood", []).append(neighbourhood)
    hotel_info_result.setdefault("hotel_region", []).append(hotel_region)
    hotel_info_result.setdefault("hotel_sido", []).append(hotel_sido)

hotel_info_df = pd.DataFrame(hotel_info_result)
hotel_info_df

,hotel_id,hotel_name,hotel_grade,n_reviews,avg_rating,cleanliness,service,comfort,condition,neighbourhood,hotel_region,hotel_sido
0,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
1,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
2,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
3,589827,바다마루 제주 (Badamaru Jeju),2.5,20,8.4,9.0,8.4,8.6,8.4,5.0,제주특별자치도,서귀포시
4,589827,바다마루 제주 (Badamaru Jeju),2.5,20,8.4,9.0,8.4,8.6,8.4,5.0,제주특별자치도,서귀포시
...,...,...,...,...,...,...,...,...,...,...,...,...
9022,507669,호텔 토스카나 (Hotel Toscana),4.0,608,9.2,9.2,9.2,9.4,9.2,7.8,제주특별자치도,서귀포시
9023,507669,호텔 토스카나 (Hotel Toscana),4.0,608,9.2,9.2,9.2,9.4,9.2,7.8,제주특별자치도,서귀포시
9024,718356256,평창펜션하늘숲 (Pyeongchang The Sky Forest),3.0,3,7.4,8.6,8.0,6.6,7.4,6.0,강원도,평창군
9025,718356256,평창펜션하늘숲 (Pyeongchang The Sky Forest),3.0,3,7.4,8.6,8.0,6.6,7.4,6.0,강원도,평창군


## User review 수집하기

In [410]:
hotel_id = data[0]['data']['body']['pdpHeader']['hotelId']
review_list = data[0]['data']['body']["reviewContent"]['reviews']["hermes"]['groups'][0]['items']

In [411]:
reviews = {}
for idx, hotel in enumerate(data[:1]):
#     print(f"{idx+1}/{len(data)}째 페이지 수집중", end='\r')
    hotel_id = hotel['data']['body']['pdpHeader']['hotelId']
    key_list = ['itineraryId', 'tripType', 'reviewDate', 'rating', 'description']
    try:    
        review_list = hotel['data']['body']["reviewContent"]['reviews']["hermes"]['groups'][0]['items']
    except Exception as e:
        print(e)
        print("리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.", end="\n\n")
    reviews['hotel_id'] = hotel_id
    for key in key_list:
        for review in review_list:
            reviews.setdefault(key, []).append(review[key])
review_df = pd.DataFrame(reviews)
review_df

,hotel_id,itineraryId,tripType,reviewDate,rating,description
0,1041793024,72009037635503,family,2020년 11월 20일,10.0,주차가 객실당 한칸씩 차고가 있는게 굉장히 특이했으며 가장 편리했음.\n간혹 차에 ...
1,1041793024,9202656047317,family,2020년 11월 14일,10.0,1. 오션뷰\n - 남해안의 어느 숙박업소가 그렇듯이 오션뷰가 좋습니다. 이 호텔...
2,1041793024,9196559102828,other,2020년 8월 11일,10.0,커플이 다녀가기에 좋은 숙소. 프라이빗함과 뷰도 갖춘 좋은 곳
3,1041793024,9201554317978,other,2020년 8월 10일,8.0,"위치와 뷰는 정말 좋습니다. 개별주차장이 생각보다 쓰기 좋았습니다. 다만, 호텔보다..."
4,1041793024,9196473456283,family,2020년 8월 7일,8.0,스탠다드 이용했어요\n방마다 차고가 있어서 편하게 짐 옮겼네요\n제가 이용한 방에 ...
5,1041793024,71000309099267,family,2019년 9월 12일,10.0,일단 깨끗하고 무엇보다주차한 곳에서 바로 룸\n으로 연결된다는 것이 넘 좋았어요~~...
6,1041793024,9201194045923,family,2020년 7월 15일,10.0,조용하고 가족과 함께 2박3일 편하게 여행하고 왔네요.
7,1041793024,9196050465566,other,2020년 6월 26일,10.0,만족합니다.
8,1041793024,9195718125010,other,2020년 4월 17일,4.0,공사장 소음과 작업자들의 분주한 소음으로 새벽부터 잠을 설쳤습니다.\n공사가 있다는...
9,1041793024,9206508579578,family,2020년 2월 21일,8.0,막 넓진않고 세식구 하루 자긴 괜춘해요^^


## Hotel_info  수집과 User_review 수집 동시 수행

In [416]:
hotel_info_result = {}
all_reviews = []
for idx, hotel in enumerate(data):
    print(f"{idx+1}/{len(data)}째 페이지 수집중", end="\r")
    hotel_id = hotel['data']['body']['pdpHeader']['hotelId']
    hotel_name = hotel['data']['body']['propertyDescription']['name']
    hotel_grade = hotel['data']['body']['propertyDescription']['starRating']
    n_reviews = hotel['data']['body']['reviewContent']['overall']['totalCount']
    avg_rating = hotel['data']['body']['reviewContent']['overall']['rating']
    cleanliness = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('cleanliness', 0.0)
    service = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('service', 0.0)
    comfort = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('comfort', 0.0)
    condition = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('condition', 0.0)
    neighbourhood = hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('neighbourhood', 0.0)
    hotel_sido = hotel['data']['body']['propertyDescription']['address']['locality']
    hotel_region = hotel['data']['body']['propertyDescription']['address'].get('region') if hotel_sido != '서울특별시' else '서울특별시'
    

    hotel_info_result.setdefault("hotel_id", []).append(hotel_id)
    hotel_info_result.setdefault("hotel_name", []).append(hotel_name)
    hotel_info_result.setdefault("hotel_grade", []).append(hotel_grade)
    hotel_info_result.setdefault("n_reviews", []).append(n_reviews)
    hotel_info_result.setdefault("avg_rating", []).append(avg_rating)
    hotel_info_result.setdefault("cleanliness", []).append(cleanliness)
    hotel_info_result.setdefault("service", []).append(service)
    hotel_info_result.setdefault("comfort", []).append(comfort)
    hotel_info_result.setdefault("condition", []).append(condition)
    hotel_info_result.setdefault("neighbourhood", []).append(neighbourhood)
    hotel_info_result.setdefault("hotel_region", []).append(hotel_region)
    hotel_info_result.setdefault("hotel_sido", []).append(hotel_sido)
    
    # 사용자 리뷰 수집 부분
    key_list = ['itineraryId', 'tripType', 'reviewDate', 'rating', 'description']
    try:    
        review_list = hotel['data']['body']["reviewContent"]['reviews']["hermes"]['groups'][0]['items']
    except Exception as e:
        print(e)
        print("리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.", end="\n\n")
    reviews = {}
    reviews['hotel_id'] = hotel_id
    for key in key_list:
        for review in review_list:
            reviews.setdefault(key, []).append(review[key])
    all_reviews.append(pd.DataFrame(reviews))
review_df = pd.concat(all_reviews)
review_df = review_df.reset_index(drop=True)

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

...

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가 없어서 다음 호텔로 넘어갑니다.

list index out of range
리뷰 데이터가

In [417]:
hotel_info_df.head(3)

,hotel_id,hotel_name,hotel_grade,n_reviews,avg_rating,cleanliness,service,comfort,condition,neighbourhood,hotel_region,hotel_sido
0,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
1,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
2,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시


In [418]:
review_df.tail(3)

,hotel_id,itineraryId,tripType,reviewDate,rating,description
197922,1344945952,9196902905665,business,2020년 9월 14일,8.0,
197923,1344945952,9201269638204,business,2020년 7월 13일,2.0,
197924,1344945952,9195821601150,with-friends,2020년 5월 22일,10.0,


In [ ]:
review_df[review_df['hotel_id']==1041793024]

In [ ]:
# csv 파일로 저장
all_reviews_df.to_csv('호텔스닷컴리뷰.csv', encoding='utf-8', index=False)